## Introduction to Computational Social Science methods with Python

### Natural Language Processing - Topic Modeling

<div class='alert alert-block alert-success'>
<b>In this Python notebook</b>, 

In this Python notebook, we will explore how to perform topic modeling using Latent Dirichlet Allocation (LDA) on thr preprocessed dataset of news articles. Topic modeling is a critical task in natural language processing (NLP), which involves discovering latent topics in a corpus of text data. LDA is a popular topic modeling technique that represents documents as mixes of topics, where each topic is a probability distribution over words in the vocabulary.

By the end of this notebook, you will have a basic understanding of how to perform topic modeling using LDA on text data, and how to analyze and visualize the discovered topics for insights and understanding. Let's get started!
</div>

## A. Latent Dirichlet Allocation (LDA)

**Latent Dirichlet Allocation (LDA)** is a generative probabilistic model which is generally used for topic modeling. 

**Topic modeling** is a technique in natural language processing (NLP) used to uncover underlying themes in a large corpus of text. The goal of topic modeling is to identify the most important topics in a collection of documents and to extract the most important words and phrases associated with each topic. There are several popular topic modeling algorithms, including Latent Dirichlet Allocation (LDA) and Non-Negative Matrix Factorization (NMF). These algorithms are **unsupervised**, meaning that they do not require labeled data. On the contrary, they are based on the assumption that words that frequently occur together in a document are likely to be associated with the same topic.

Topic modeling typically involves the following steps:

- Preprocessing the data: The text data is cleaned and preprocessed to remove unnecessary information such as stop words, punctuation, and special characters.
- Vectorization: the preprocessed text data is converted into a numerical vector representation, such as a bag-of-words or TF-IDF (Term Frequency-Inverse Document Frequency) matrix.
- Model training: a topic modeling algorithm is applied to the vectorized text data to identify the main topics or themes. This includes identifying the words or phrases most strongly associated with each topic.
- Theme Interpretation: the resulting themes are interpreted by examining the most representative words or phrases for each theme. This may involve manually examining the most important words for each theme or using automated techniques to summarize the themes.

LDA assumes that each document is a mixture of different topics, and each topic is a probability distribution over words. The algorithm takes a collection of documents as input and returns a set of topics, each represented by a probability distribution over words.

The basic idea behind LDA is that each document is represented as a distribution of topics and each topic is represented as a distribution of words. The algorithm assumes that the documents are generated in the following way:

- For each document, a distribution of topics is selected from a Dirichlet distribution.
- For each word in the document:
    - Select a topic from the distribution of topics of the document.
    - Select a word from the distribution of words for the selected topic.

The algorithm iteratively learns the topic distributions for each document and the word distributions for each topic by maximizing the probability of the observed data. This optimization is usually done using the expectation maximization algorithm (EM).

Once the algorithm has converged, it returns a set of topics, each represented by a distribution over words. These themes can be interpreted by examining the most likely words for each theme. 

Here we present an example of LDA applied to a set of documents. We will use th news articles that we preprocessed previous notebook. Let's import the TF-IDF matrix that we previously extracted from this corpus:

In [6]:
import pandas as pd
import numpy as np 
import pickle as pkl

# import raw dataset 
news = pd.read_csv("../data/news_subset.csv")

# import also the dictionary, the preprocessed corpus, and the BoW and TF-IDF matrices
with open("./output/tf_idf_gensim.pkl", "rb") as file:
    tf_idf = pkl.load(file)

with open("./output/dict_gensim.pkl", "rb") as file:
    dct = pkl.load(file)

with open("./output/corpus.pkl", "rb") as file:
    corpus = pkl.load(file)

with open("./output/document_term_matrix.pkl", "rb") as file:
    corpus_bow = pkl.load(file)

Then, we run LDA on the document-term matrix. We use the Gensim implementation of LDA:

In [11]:
import gensim
from gensim.models import LdaModel

# train an LDA model on the TF-IDF corpus
num_topics = 10
lda_model = LdaModel(corpus_bow,
                     id2word=dct, 
                     num_topics=num_topics, 
                     passes=10)

We can visualize the extracted topics by looking at the 10 most common words of each topic:

In [12]:
# print the topics and associated keywords
for topic in lda_model.print_topics():
    print(topic)

(0, '0.041*"photo" + 0.016*"look" + 0.016*"wed" + 0.012*"dress" + 0.010*"style" + 0.009*"fashion" + 0.009*"recip" + 0.008*"best" + 0.007*"wear" + 0.007*"celebr"')
(1, '0.012*"time" + 0.011*"way" + 0.010*"peopl" + 0.010*"dont" + 0.010*"life" + 0.009*"like" + 0.009*"know" + 0.009*"thing" + 0.009*"need" + 0.008*"kid"')
(2, '0.034*"divorc" + 0.018*"hotel" + 0.013*"patient" + 0.010*"chanc" + 0.009*"hospit" + 0.009*"car" + 0.007*"notic" + 0.007*"seat" + 0.006*"airlin" + 0.006*"rape"')
(3, '0.015*"video" + 0.012*"love" + 0.009*"coupl" + 0.009*"day" + 0.008*"marriag" + 0.008*"gay" + 0.007*"week" + 0.007*"date" + 0.007*"photo" + 0.006*"emot"')
(4, '0.016*"school" + 0.012*"sex" + 0.012*"state" + 0.012*"student" + 0.009*"program" + 0.008*"univers" + 0.008*"colleg" + 0.007*"awar" + 0.007*"california" + 0.006*"offici"')
(5, '0.016*"photo" + 0.013*"new" + 0.012*"citi" + 0.010*"world" + 0.010*"sleep" + 0.007*"best" + 0.007*"music" + 0.007*"hair" + 0.007*"teen" + 0.006*"year"')
(6, '0.010*"game" + 0.0

To assess how good are the topics extracted by the model, we can use different metrics such as the coherence score and perplexity.

**Coherence score** is a measure of how coherent the topics generated by a topic model are, based on the co-occurrence of words in the corpus. It is often used as an evaluation metric for topic models, in addition to perplexity.
The coherence score is typically based on the top N words in each topic, and measures the similarity between pairs of words in the same topic. There are different ways to define the coherence score, but one common approach is to use the Pointwise Mutual Information (PMI) between pairs of words.

PMI measures the degree of association between two words, based on the probability of their co-occurrence in the corpus relative to their individual probabilities. A high PMI score indicates that the two words are strongly associated and likely to appear together in the same context.

The coherence score is calculated as the average PMI score over all pairs of words in each topic, and then averaged over all topics in the model. A higher coherence score indicates that the topics are more coherent and contain more related and meaningful words.
Coherence score is often used in combination with perplexity to evaluate the quality of a topic model. Perplexity measures how well the model fits the data, while coherence measures how well the topics generated by the model make sense in terms of the co-occurrence of words in the corpus.

We can also use coherence score to calibrate the number of topics to use, by training several models and comparing them to each other using the coherence score. The model with highest coherence score is then used. 



In [13]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=dct)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score with {0} topics: {1}'.format(num_topics, coherence_lda))



Coherence Score with 10 topics: 0.38081833732024517


In [ ]:
#scores = []
#for num_topics in np.arange(5, 20):

    # fit LDA model
    #lda_model = LdaModel(corpus_bow, id2word=dct, num_topics=num_topics)

    # compute Coherence Score
    #coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=dct)
    #coherence_lda = coherence_model_lda.get_coherence()
    #print('\nCoherence Score with {0} topics: {1}'.format(num_topics, coherence_lda))

    #scores.append(coherence_lda)

#import matplotlib.pyplot as plt
#plt.plot(np.arange(5, 20), scores, marker="o")
#plt.xlabel("Number of topics")
#plt.ylabel("Coherence Score")

#print("Max coherence with: {0} topics".format(np.arange(5, 20)[np.argmax(scores)]))

Finally, using the package pyLDAvis we can explore the extracted topis:

In [14]:
import pyLDAvis
import pyLDAvis.gensim
# !pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_bow, dct)
vis

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.236575  0.112515       1        1  28.934584
7      0.118351  0.245690       2        1  17.746715
3      0.150343 -0.098984       3        1   9.164249
0      0.128414 -0.196147       4        1   8.803223
5      0.098595 -0.136481       5        1   8.141879
9     -0.039642  0.101438       6        1   6.651046
6     -0.059876 -0.093957       7        1   6.098519
4     -0.172997  0.186031       8        1   5.374619
8     -0.209087 -0.040853       9        1   5.088601
2     -0.250676 -0.079252      10        1   3.996566, topic_info=              Term         Freq        Total Category  logprob  loglift
179          photo  4306.000000  4306.000000  Default  30.0000  30.0000
27           trump  3923.000000  3923.000000  Default  29.0000  29.0000
443          women  1968.000000  1968.000000  Default  28.0000  28.0000
592   donald_trump  1983.000000  1983.000000  Default  27.0000  27.0000
437         divorc  1050.000000  1050.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
623     washington   171.464287   278.134045  Topic10  -5.2021   2.7360
943         travel   156.376387  1218.249011  Topic10  -5.2942   1.1668
2518          wife   130.923702   250.662614  Topic10  -5.4718   2.5702
1898        turkey   127.730703   206.306534  Topic10  -5.4965   2.7403
5775         arriv   125.815222   213.334515  Topic10  -5.5116   2.6917

[525 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1534      6  0.994204     abil
2075      5  0.993328  absolut
4545      4  0.997987   accept
1213      3  0.207922    actor
1213      7  0.769622    actor
...     ...       ...      ...
536       3  0.446759  yearold
536       4  0.142583  yearold
536       7  0.142583  yearold
536       9  0.053865  yearold
1903      6  0.995444     yoga

[797 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 4, 1, 6, 10, 7, 5, 9, 3])